<a href="https://colab.research.google.com/github/Luke-zm/coursera_learning/blob/main/udemy_pyomo/pyomo_ip_tut2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 2

A firm thar produces automobile seats manufactures 3 seat types on 2 different production lines.  
Up to 30 workers can be used at the same time on each production line to produce any seat type.  
Each worker is paid \$400 per week on production line 1, and \$600 per week on production line 2.   
1 week of production on line 1 cost \$1000, and cost \$2000 on line 2.  
Below table shows the number of seats by each worker on a production line.  

|Line|Seat1|Seat2|seat3|  
|----|-----|-----|-----|  
|  1 | 20  |  30 |  40 |  
|  2 | 50  |  35 |  45 |   

The weekly demand of seat:
seat1: 120.  
seat2: 150.  
seat3: 200.  

Use Integer Linear Programming to minimize the total cost of production plan.  




## Mathematical model

Let the demand of each seat type be D_i.  
Let the number of each type of seat produced on each line be X_i,j.  
Let the cost of each worker be W_j.  
Let the cost of each line be L_j.  
Let the decsion of starting a line be S_j.  
Let the big M for each line be M_j.  
Let the number of each worker on each line be N_j.   
Let i be in the range 1 to 3.   
Let j be in the range 1 to 2.  

$$  
\\objective~ function:  
\\min(\sum_{j=1}(N_j \times W_j) + \sum_{j=1}(S_j\times L_j))
\\s.t  
\\\sum(X_{i,j}\times N_j)\geq D_i
\\N_j \leq S_j\times M_j  
$$  

In [2]:
# Install denpendencies
!pip install pyomo
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [3]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory